In [29]:
import numpy as np

def random_20d_lattice_basis(low=-100, high=101):
    while True:
        basis = np.random.randint(low, high, size=(10, 10))
        if np.linalg.matrix_rank(basis) == 10:
            return basis

# 示例：生成一个二十维格基
basis = random_20d_lattice_basis()
print(basis)
# 原始格基中最短的向量
shortest_vector = min(basis, key=lambda v: np.linalg.norm(v))
print("原始格基中最短的向量：", shortest_vector)
print("最短向量的范数：", np.linalg.norm(shortest_vector))

[[  32   70    7   70  -59  -35  -22    7  -38   49]
 [   8  -84  -39  -21  -30   53   62   58   88  -30]
 [ -77   73  -14  -40  -43   74   16  -21   24 -100]
 [  15  -52  -29  -68   89   94   32   18  -18  -27]
 [ -57   61    6   10  -86  -36   19   73   60  -23]
 [  25  -59   30   -1   71  -19   80   96   69   46]
 [  -9   99  -34  -33    4   40   -1   44  -68  -98]
 [ -23   -6   94  -58   46   86  -28   96   58   97]
 [  32  -71  -96  -58   60   86   80  -51  -87  -40]
 [  -9   88   23   48   49  -20  -14  -97   26   98]]
原始格基中最短的向量： [ 32  70   7  70 -59 -35 -22   7 -38  49]
最短向量的范数： 141.2692464763651


In [30]:
def gram_schmidt(B):
    n = B.shape[0]
    B_star = np.zeros_like(B, dtype=np.float64)
    mu = np.zeros((n, n), dtype=np.float64)
    for i in range(n):
        B_star[i] = B[i]
        for j in range(i):
            mu[i, j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
            B_star[i] -= mu[i, j] * B_star[j]
    return B_star, mu

def lll_reduction(B, delta=0.75):
    n = B.shape[0]
    B = B.astype(np.int64)
    k = 1
    B_star, mu = gram_schmidt(B)
    while k < n:
        for j in range(k-1, -1, -1):
            c = int(round(mu[k, j]))
            if c != 0:
                B[k] -= c * B[j]
                B_star, mu = gram_schmidt(B)
        if np.dot(B_star[k], B_star[k]) >= (delta - mu[k, k-1]**2) * np.dot(B_star[k-1], B_star[k-1]):
            k += 1
        else:
            B[[k, k-1]] = B[[k-1, k]]
            B_star, mu = gram_schmidt(B)
            k = max(k-1, 1)
    return B

# 示例：生成一个随机二十维格基并进行LLL约减
basis = random_20d_lattice_basis()
print("原始二十维格基：\n", basis)
reduced_basis = lll_reduction(basis)
print("LLL约减后的二十维格基：\n", reduced_basis)

# 打印 LLL 规约后的第一个向量
print("LLL规约后的第一个向量：", reduced_basis[0])
# 打印 LLL 规约后的第一个向量的范数
print("LLL规约后的第一个向量的范数：", np.linalg.norm(reduced_basis[0]))

原始二十维格基：
 [[ 60 -44   4  12  24 -61  -9  77   5  34]
 [ 44 -71 -75  19 -25 -14  55  70  60  77]
 [ 29 -24 -47 -90 -14  24 -43 -15  89 -30]
 [-71  83  44   9  73 -94 -16  99 -29 -47]
 [-93  10  37 -23  88  41 -59  55  81 -34]
 [-59 -59  -3 100 -92 -97 -62  -2 -71 -52]
 [ 52  60 -15 -43 -37   4 -71 -24 -89 -75]
 [ 89 100 -28  -2  -6 -35 -74   6  49  23]
 [-59 -15   8  73   7  13  60  12 -92  -5]
 [ 64 -73  60  39  40  46 -54   1  73  -6]]
LLL约减后的二十维格基：
 [[ -30  -39  -39  -17   -7   37   17   -3   -3  -35]
 [  30  -83  -35   -5   17  -24    8   74    2   -1]
 [  -7  -11  -12    0   66  -12  -33   31   77  -15]
 [  14   12  -40   24  -42   10   47   -4   58   78]
 [  66   26    4  -73  -73   -1  -27  -43   15   20]
 [  10   10  -41  -47  -40   43  -42   70  -25  -17]
 [ -86   21   49  -23   22   53  -26   24    4  -19]
 [ -16   -6   37   55   -2   11   27   62   22    5]
 [ -75   21   11  -51   26  -64   33  -20    5  -80]
 [-115  -20  -31   46 -110  -45  -68  -44  -31    2]]
LLL规约后的第一个向量：